# Plot bicycle network results
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook takes the results from 03_poi_based_generation and 04_connect_clusters and 05_analyze_results, and plots them.

NOT YET DONE.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-07-08

## Preliminaries

### Imports and magics

In [32]:
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
import osmnx as ox
import copy
import math
from haversine import haversine
import pprint
import sys
pp = pprint.PrettyPrinter(indent=4)

%matplotlib inline
import pandas as pd
from matplotlib import cm

import watermark
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [33]:
%watermark -n -v -m -g -iv

numpy     1.18.4
igraph    0.8.2
watermark 2.0.2
pandas    1.0.3
osmnx     0.13.0
Tue Jul 07 2020 

CPython 3.8.2
IPython 7.14.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit
Git hash   :


### Parameters

In [34]:
placeid = "budapest"
prune_measure = "betweenness"
cutoff = 0.8 # How many clusters should be considered (covering cutoff fraction of total length)

prune_quantiles = [x/20 for x in list(range(1, 21))] # The quantiles where the GT should be pruned using the prune_measure
prune_measures = {"betweenness": "BWq", "closeness": "CCq"}
datapath = "../data/"
outpath = "../output/"
ymirrored = False